In [149]:
import pandas as pd
from ast import literal_eval
from statistics import mean
import pandas as pd
import math
from statistics import mean
from tqdm import tqdm
import os
import ast
import numpy as np

In [150]:
#Match them back to their models
ref = "data/full_responses_french_v3_annots.csv"
data_ref = pd.read_csv(ref, encoding = 'UTF-8')

<h1>Output of STEPS 1 & 2: Filter and Top-3</h1>

In [151]:
def prep_batch(batch):
    for col in batch.columns:
        if col == 'context':
            batch[col] = list(batch[col].values)

        else:
            batch[col] = batch[col].apply(lambda x: literal_eval(x) if "[" in x else x)
    
    return batch.to_dict()

In [152]:
annotators = ['Audrey', 'Cindy', 'Tiphaine']

In [153]:
df = pd.read_csv("data/final_results_filter_DEV_NEW_DPO_clean.csv", encoding='UTF-8')
df = prep_batch(df)

pd.DataFrame(df).head(n=10)

,context,annotators,eliminated,filtered,top-3
0,Customer(C): Euh ouais bah euh ouais demain di...,"[Audrey, Cindy]","[[Oui, je vais vérifier si [Ville d'un centre]...","[[D'accord, je vais vérifier ça pour vous. Si ...","[[Super, je vais donc organiser une installati..."
1,Customer(C): D'accord et comment ça se passe i...,"[Audrey, Tiphaine]","[[Bien sûr, nous pouvons organiser une visite ...",[[Le centre de réparation automobile est situé...,[[Je peux vous donner une estimation des frais...
2,Customer(C): Alors attendez une seconde une se...,"[Audrey, Tiphaine]","[[D'accord, je suis là. Pouvez-vous me dire ce...","[[Quel est le modèle de votre véhicule, madame...","[[Avez-vous trouvé la carte verte, madame?, D'..."
3,Customer(C): Ouais f r ? vous voulez le f r ? ...,"[Audrey, Tiphaine]",[[Merci pour les informations fournies. Votre ...,"[[Merci, j'ai tout noté. Pouvez-vous me donner...","[[Merci, j'ai tout noté. Pouvez-vous me donner..."
4,Customer(C): J'ai pas d'autre chose j'ai pas d...,"[Audrey, Tiphaine]","[[""Je vais essayer de vous mettre en communica...","[[D'accord, je vais vous mettre en communicati...","[[D'accord, je vais vous mettre en communicati..."
5,Customer(C): Oui d'accord j'ai pas entendu voi...,"[Audrey, Tiphaine]","[[D'accord, votre adresse est bien 12 rue Marg...","[[D'accord, merci pour les informations. Pourr...","[[""Okay, madame, je vais mettre à jour votre a..."
6,Customer(C): Ouais non [Nom de ville (Client)]...,"[Audrey, Tiphaine]","[[D'accord, je vais essayer de contacter le ce...","[[Oui, je comprends parfaitement votre situati...","[[""D'accord, je vais appeler le centre pour vo..."
7,Customer(C): Bah ah ouais ? et euh sur [Ville ...,"[Audrey, Tiphaine]","[[Bonjour, je vérifie pour vous. Oui, nous avo...","[[D'accord, je vais vérifier pour vous. Pouvez...","[[D'accord, je vais vérifier pour vous. Pouvez..."
8,Customer(C): Oui il vous faut quoi ? oui oui l...,"[Audrey, Tiphaine]","[[Madame, je suis désolé, mais il semble y avo...","[[Oui madame, je comprends. Pouvez-vous me dir...","[[Non, madame, je veux savoir si votre carte e..."
9,Customer(C): Oui bonjour madame je vous appell...,"[Audrey, Tiphaine]","[[D'accord, je vais vous laisser un peu de tem...","[[D'accord, prenez votre temps, je vais attend...","[[D'accord, prenez votre temps, je vais attend..."


In [154]:
contexts = df['context']
len(contexts)

100

In [155]:
def merge_lists(list_of_lists):
    unique = []

    for list in list_of_lists:
        for elem in list:
            if elem not in unique:
                unique.append(elem)

    return unique


def merge_tops(list_of_lists):
    final_top = []
    if len(list_of_lists)==2:
        final_top = [x for x in list_of_lists[0] if x in list_of_lists[1]]
    
    #elif len(list_of_lists)==3:
    #    final_top = [x for x in list_of_lists[0] if x in list_of_lists[1] and x in list_of_lists[2]]
    
    else:
        print('OOPS', len(list_of_lists))
    
    return final_top


In [156]:
def match_responses_to_model(context, responses, model_responses):
    model_responses_contexts = model_responses['context'].tolist()
    index_context = model_responses_contexts.index(context)

    cols = [col for col in model_responses.columns if col not in ['context']]
    row = model_responses.iloc[index_context]
    #print(row)

    models=[]
    for response in responses:
        response = response
        for col in cols:
            #print("COL: ", col)
            if isinstance(row[col], str):
                if response == row[col].strip():
                    models.append(col)
    

    return models

In [157]:
models = [x for x in data_ref.columns if x not in ["context", 'emotions', 'strats', "gt segments", "gt segment strats", 'ground truth']]
print(models)

['qwen naive planning', 'qwen response e2e', 'qwen 2 step planning', 'qwen 3 steps planning', 'llama naive planning', 'llama response e2e', 'llama 2 step planning', 'llama 3 steps planning', 'mixtral naive planning', 'mixtral response e2e', 'mixtral 2 step planning', 'mixtral 3 steps planning', 'qwen dpo response e2e', 'qwen dpo naive planning']


In [158]:
dic_per_annot_models = {'context': contexts, 'annotators': df['annotators'], 'eliminated': [], 'filtered': [], 'top-3': []}
for i in range(len(contexts)):
    ctx = contexts[i]
    dic_per_annot_models['eliminated'].append([match_responses_to_model(ctx, df['eliminated'][i][0], data_ref), match_responses_to_model(ctx, df['eliminated'][i][1], data_ref)])
    dic_per_annot_models['filtered'].append([match_responses_to_model(ctx, df['filtered'][i][0], data_ref), match_responses_to_model(ctx, df['filtered'][i][1], data_ref)])
    dic_per_annot_models['top-3'].append([match_responses_to_model(ctx, df['top-3'][i][0], data_ref), match_responses_to_model(ctx, df['top-3'][i][1], data_ref)])

In [159]:
stats_models_filter = {annot:{model: 0 for model in models} for annot in annotators}
stats_models_top3 = {annot:{model: 0 for model in models} for annot in annotators}
stats_models_eliminated = {annot:{model: 0 for model in models} for annot in annotators}


for i in range(len(contexts)):
    annots = dic_per_annot_models['annotators'][i]
    for j, annotator in enumerate(annots):

        filtered_models = dic_per_annot_models['filtered'][i][j]
        for model in filtered_models:
            stats_models_filter[annotator][model] += 1

        eliminated_models = dic_per_annot_models['eliminated'][i][j]
        for model in eliminated_models:
            stats_models_eliminated[annotator][model] += 1

        top_3_models = dic_per_annot_models['top-3'][i][j]
        for model in top_3_models:
            stats_models_top3[annotator][model] += 1

In [160]:
filter_m = {m:{2:0, 1:0, 0:0} for m in models}
top_m = {m:{2:0, 1:0, 0:0} for m in models}

for i, ctx in enumerate(contexts):
    for model in models:
        if model in dic_per_annot_models['filtered'][i][0] and model in dic_per_annot_models['filtered'][i][1]:
            filter_m[model][2] +=1

        
        elif model in dic_per_annot_models['filtered'][i][0] or model in dic_per_annot_models['filtered'][i][1]:
            filter_m[model][1] +=1
        
        else:
            filter_m[model][0] +=1


        if model in dic_per_annot_models['top-3'][i][0] and model in dic_per_annot_models['top-3'][i][1]:
            top_m[model][2] +=1

        elif model in dic_per_annot_models['top-3'][i][0] or model in dic_per_annot_models['top-3'][i][1]:
            top_m[model][1] +=1
        
        else:
            top_m[model][0] +=1

scores_models_filter = {m : (v[2] + v[1]) for m, v in zip(models, filter_m.values())}
scores_models_top3 = {m : (v[2] + v[1]) for m, v in zip(models, top_m.values())}

In [161]:
scores = {m:{'filtered': 0, 'filtered weighted':0, 'top-3': 0, 'top@1':0, 'top@2':0, 'top@3':0} for m in models}
for model in (scores_models_filter.keys()):
    v = filter_m[model]
    scores[model]['filtered'] = (v[2] + v[1]) / len(contexts)
    scores[model]['filtered weighted'] = round((1.5*v[2] + v[1]) / len(contexts),2)

In [162]:
for model in models:
    print(model, scores[model]['filtered'], scores[model]['filtered weighted'])

qwen naive planning 0.65 0.81
qwen response e2e 0.55 0.69
qwen 2 step planning 0.23 0.27
qwen 3 steps planning 0.23 0.28
llama naive planning 0.7 0.92
llama response e2e 0.67 0.81
llama 2 step planning 0.42 0.52
llama 3 steps planning 0.51 0.67
mixtral naive planning 0.63 0.85
mixtral response e2e 0.72 0.92
mixtral 2 step planning 0.29 0.34
mixtral 3 steps planning 0.25 0.3
qwen dpo response e2e 0.03 0.04
qwen dpo naive planning 0.04 0.05


In [163]:
scores_models_filter = {k: v for k, v in sorted(scores_models_filter.items(), reverse= True, key=lambda item: item[1])}

print("MOST FILTERED MODELS\n")
for model in (scores_models_filter.keys()):
    v = filter_m[model]
    print(model, '\t\t', v[2] + v[1], "\t-   SELECTED TWICE: ", v[2], ', SELECTED ONCE: ', v[1], ', NOT SELECTED: ', v[0])

MOST FILTERED MODELS

mixtral response e2e 		 72 	-   SELECTED TWICE:  40 , SELECTED ONCE:  32 , NOT SELECTED:  28
llama naive planning 		 70 	-   SELECTED TWICE:  43 , SELECTED ONCE:  27 , NOT SELECTED:  30
llama response e2e 		 67 	-   SELECTED TWICE:  27 , SELECTED ONCE:  40 , NOT SELECTED:  33
qwen naive planning 		 65 	-   SELECTED TWICE:  33 , SELECTED ONCE:  32 , NOT SELECTED:  35
mixtral naive planning 		 63 	-   SELECTED TWICE:  45 , SELECTED ONCE:  18 , NOT SELECTED:  37
qwen response e2e 		 55 	-   SELECTED TWICE:  27 , SELECTED ONCE:  28 , NOT SELECTED:  45
llama 3 steps planning 		 51 	-   SELECTED TWICE:  31 , SELECTED ONCE:  20 , NOT SELECTED:  49
llama 2 step planning 		 42 	-   SELECTED TWICE:  19 , SELECTED ONCE:  23 , NOT SELECTED:  58
mixtral 2 step planning 		 29 	-   SELECTED TWICE:  9 , SELECTED ONCE:  20 , NOT SELECTED:  71
mixtral 3 steps planning 		 25 	-   SELECTED TWICE:  11 , SELECTED ONCE:  14 , NOT SELECTED:  75
qwen 2 step planning 		 23 	-   SELECTED TW

In [164]:
scores_models_top3 = {k: v for k, v in sorted(scores_models_top3.items(), reverse= True, key=lambda item: item[1])}

print("MOST SELECTED MODELS AS TOP-3\n")
for model in (scores_models_top3.keys()):
    v = top_m[model]
    print(model, '\t\t', v[2] +v[1], "\t-   SELECTED TWICE: ", v[2], ', SELECTED ONCE: ', v[1], ', NOT SELECTED: ', v[0])

MOST SELECTED MODELS AS TOP-3

llama naive planning 		 58 	-   SELECTED TWICE:  25 , SELECTED ONCE:  33 , NOT SELECTED:  42
mixtral naive planning 		 53 	-   SELECTED TWICE:  28 , SELECTED ONCE:  25 , NOT SELECTED:  47
mixtral response e2e 		 48 	-   SELECTED TWICE:  25 , SELECTED ONCE:  23 , NOT SELECTED:  52
qwen naive planning 		 46 	-   SELECTED TWICE:  18 , SELECTED ONCE:  28 , NOT SELECTED:  54
llama response e2e 		 40 	-   SELECTED TWICE:  20 , SELECTED ONCE:  20 , NOT SELECTED:  60
qwen response e2e 		 35 	-   SELECTED TWICE:  11 , SELECTED ONCE:  24 , NOT SELECTED:  65
llama 3 steps planning 		 34 	-   SELECTED TWICE:  11 , SELECTED ONCE:  23 , NOT SELECTED:  66
llama 2 step planning 		 29 	-   SELECTED TWICE:  8 , SELECTED ONCE:  21 , NOT SELECTED:  71
mixtral 2 step planning 		 16 	-   SELECTED TWICE:  4 , SELECTED ONCE:  12 , NOT SELECTED:  84
mixtral 3 steps planning 		 10 	-   SELECTED TWICE:  4 , SELECTED ONCE:  6 , NOT SELECTED:  90
qwen 2 step planning 		 9 	-   SELECT

In [165]:
top_3_ranked = {1:{m:0 for m in models}, 2:{m:0 for m in models}, 3:{m:0 for m in models}, 4:{m:0 for m in models}}

for tops in dic_per_annot_models['top-3']:
    for top in tops:
        for i in range(len(top)):
            top_3_ranked[i+1][top[i]] +=1

In [166]:
for m in models: 
    v = []
    for i in range(1,4):
        v.append(top_3_ranked[i][m])
    scores[m]['top-3'] = round((v[0] + v[1] + v[2]) / len(contexts), 2)

    if scores[m]['top-3'] > 0:
        scores[m]['top@1'] = round(((v[0] / scores[m]['top-3'])/len(contexts))*scores[m]['top-3'],2) 
        scores[m]['top@2'] = round((((v[1]) / scores[m]['top-3'])/len(contexts))*scores[m]['top-3'],2)
        scores[m]['top@3'] = round((((v[2]) / scores[m]['top-3'])/len(contexts))*scores[m]['top-3'],2)

    else:
        scores[m]['top-1'] = 0
        scores[m]['top-2'] = 0

In [167]:
for i in top_3_ranked.keys():
    print(' ---- SELECTED AS ', i)
    topi = {k: v for k, v in sorted(top_3_ranked[i].items(), key=lambda item: item[1], reverse=True)}
    for k, v in topi.items():
        if v >0:
            print(k, v)

 ---- SELECTED AS  1
qwen naive planning 64
llama naive planning 40
qwen response e2e 32
llama response e2e 24
llama 2 step planning 13
mixtral naive planning 10
qwen 2 step planning 4
mixtral response e2e 4
qwen 3 steps planning 2
llama 3 steps planning 2
mixtral 3 steps planning 1
 ---- SELECTED AS  2
llama naive planning 38
mixtral naive planning 34
llama response e2e 25
llama 3 steps planning 25
llama 2 step planning 15
qwen response e2e 14
mixtral response e2e 14
qwen 2 step planning 6
qwen 3 steps planning 6
mixtral 2 step planning 4
mixtral 3 steps planning 1
qwen dpo response e2e 1
 ---- SELECTED AS  3
mixtral response e2e 55
mixtral naive planning 37
llama 3 steps planning 18
mixtral 2 step planning 15
llama response e2e 10
mixtral 3 steps planning 9
llama 2 step planning 7
llama naive planning 5
qwen 3 steps planning 3
qwen dpo naive planning 2
qwen dpo response e2e 1
 ---- SELECTED AS  4
mixtral 3 steps planning 3
llama 2 step planning 2
llama response e2e 1
mixtral 2 step p

In [168]:
import numpy as np

mtx = np.zeros((len(models), len(scores[models[0]])))

for i, model in enumerate(models):
    for j, score in enumerate(list(scores[model].keys())):
        mtx[i][j] = scores[model][score]

In [169]:
panda_df = pd.DataFrame(data = mtx,  
                        index = models,  
                        columns = list(scores[model].keys())
) 
print(panda_df)

                          filtered  filtered weighted  top-3  top@1  top@2  \
qwen naive planning           0.65               0.81   0.64   0.64   0.00   
qwen response e2e             0.55               0.69   0.46   0.32   0.14   
qwen 2 step planning          0.23               0.27   0.10   0.04   0.06   
qwen 3 steps planning         0.23               0.28   0.11   0.02   0.06   
llama naive planning          0.70               0.92   0.83   0.40   0.38   
llama response e2e            0.67               0.81   0.59   0.24   0.25   
llama 2 step planning         0.42               0.52   0.35   0.13   0.15   
llama 3 steps planning        0.51               0.67   0.45   0.02   0.25   
mixtral naive planning        0.63               0.85   0.81   0.10   0.34   
mixtral response e2e          0.72               0.92   0.73   0.04   0.14   
mixtral 2 step planning       0.29               0.34   0.19   0.00   0.04   
mixtral 3 steps planning      0.25               0.30   0.11   0

In [170]:
filter_scores = scores

In [171]:
#SAVE IN TXT FILE
file_generated = "filter_top_results"
print('Saving results...')
with open("results/"+file_generated+".txt", "w") as f:
  print(panda_df, file=f)

Saving results...


<h1>Output of STEPS 3: Fine-Grained Socio-Emotional Annotation</h1>

<h2>Individual Scores</h2>

In [172]:
annotators = ['Amira', 'Cindy', 'Hayoung', 'Tiphaine']

In [173]:
path = "data/human_eval_socemo_annot.csv"
data = pd.read_csv(path, encoding='UTF-8')

dic_of_ratings = {x:list(data[x]) for x in data.columns}

In [174]:
cols = ['model', 'response', 'context']
ratings = ['annotated_strategy',"usefulness","fluency","emotion","emotion_adequate","strategy_adequate","role_consistency", 'global_consistency']

all_cols = cols + ratings

In [175]:
models = ['qwen naive planning','qwen response e2e','llama naive planning','llama response e2e','mixtral naive planning',
 'mixtral response e2e','ground truth']
 
models
 

['qwen naive planning',
 'qwen response e2e',
 'llama naive planning',
 'llama response e2e',
 'mixtral naive planning',
 'mixtral response e2e',
 'ground truth']

In [176]:
dic_model_criteria = {m:{h:[] for h in list(dic_of_ratings.keys()) if h != 'model'} for m in models}

for i in range(len(dic_of_ratings['model'])):
    m = dic_of_ratings['model'][i]
    
    for col in list(dic_model_criteria[m].keys()):
        dic_model_criteria[m][col].append(dic_of_ratings[col][i])

In [177]:
dic_model_criteria[m].keys()

dict_keys(['id', 'response', 'context', 'annotated_strategy', 'usefulness', 'fluency', 'emotion', 'emotion_adequate', 'strategy_adequate', 'role_consistency', 'naturalness', 'gt emotion', 'gt segment strats', 'gt segments', 'pred labels'])

In [178]:
from textdistance import levenshtein
import distance

for m in models:
    ref = [literal_eval(x) for x in dic_model_criteria[m]['gt segment strats']]
    annot =  dic_model_criteria[m]['pred labels']
    dic_model_criteria[m]['lev strat'] = round(mean([levenshtein.normalized_similarity(a,b) for a,b in zip(ref, annot)]),2)
    dic_model_criteria[m]['jacc strat'] = round(mean([distance.jaccard(a,b) for a,b in zip(ref, annot)]),2)

In [179]:
from sklearn.metrics import accuracy_score,f1_score


for m in models:
    ref = dic_model_criteria[m]['gt emotion']
    annot =  dic_model_criteria[m]['emotion']
    dic_model_criteria[m]['emo f1'] = round(f1_score(annot,ref, average='weighted'),2)
    #dic_model_criteria[m]['emo f1'] = round(accuracy_score(annot,ref),2)

In [180]:
dic_model_criteria[m].keys()

cols = ['usefulness', 'fluency', 'naturalness', 'emo f1', 'emotion_adequate', 'strategy_adequate', 'lev strat', 'jacc strat', 'role_consistency']

In [181]:
import numpy as np

scores_all = {m:{} for m in models}
mtx = np.zeros((len(models), len(cols)))

for i, m in enumerate(models):
    for j, col in enumerate(cols):
        if isinstance(dic_model_criteria[m][col], list):
            mtx[i][j] = round(mean(dic_model_criteria[m][col]),2)
            scores_all[m][col] = round(mean(dic_model_criteria[m][col]),2)
        
        else:
            mtx[i][j] = round(dic_model_criteria[m][col],2)
            scores_all[m][col] = round(dic_model_criteria[m][col],2)


        if col in ['usefulness',  'fluency',  'emotion_adequate', 'strategy_adequate', 'naturalness']:
            mtx[i][j] = round(mtx[i][j] /3,2)
            scores_all[m][col] = round(scores_all[m][col] /3,2)


In [182]:
panda_df = pd.DataFrame(data = mtx,  
                        index = models,  
                        columns = cols) 
print(panda_df)

                        usefulness  fluency  naturalness  emo f1  \
qwen naive planning           0.92     0.92         0.78    0.92   
qwen response e2e             0.70     0.88         0.72    0.73   
llama naive planning          0.88     0.88         0.82    0.74   
llama response e2e            0.87     0.92         0.78    0.95   
mixtral naive planning        0.83     0.90         0.83    0.86   
mixtral response e2e          0.85     0.90         0.70    0.90   
ground truth                  0.87     0.92         0.88    0.68   

                        emotion_adequate  strategy_adequate  lev strat  \
qwen naive planning                 1.00               0.97       0.09   
qwen response e2e                   0.97               0.98       0.12   
llama naive planning                0.95               0.95       0.12   
llama response e2e                  0.95               0.95       0.11   
mixtral naive planning              0.95               0.92       0.08   
mixtral res

In [183]:
human_scores = scores_all

In [184]:
#SAVE IN data/ FILE
file_generated = "all_criteria"
print('Saving results...')
with open("results/"+file_generated+".txt", "w") as f:
  print(panda_df, file=f)

Saving results...


<h2>Aggregated Scores</h2>

In [185]:
dic_of_scores = {'id':dic_of_ratings['id'], 'model':dic_of_ratings['model'], 'response':dic_of_ratings['response'], 'total':[], 'logical':[], 'emotional':[], 'social':[]}

for i in range(len(dic_of_ratings['id'])):
    logical = (dic_of_ratings['usefulness'][i]/3 + dic_of_ratings['fluency'][i]/3  + dic_of_ratings['naturalness'][i] /3) / 3
    emo = dic_of_ratings['emotion_adequate'][i] /3
    social = (dic_of_ratings['strategy_adequate'][i]/3 + dic_of_ratings['role_consistency'][i] ) / 2
    total = (logical + emo + social) / 3

    dic_of_scores['total'].append(total)
    dic_of_scores['logical'].append(logical)
    dic_of_scores['emotional'].append(emo)
    dic_of_scores['social'].append(social)

In [186]:
total = round(mean(dic_of_scores['total']),2)
logical = round(mean(dic_of_scores['logical']),2)
emo = round(mean(dic_of_scores['emotional']),2)
social = round(mean(dic_of_scores['social']),2)


print('AVERAGE GLOBAL TOTAL SCORE: ', total)
print('')
print('AVERAGE GLOBAL LOGICAL SCORE: ', logical)    
print('AVERAGE GLOBAL EMO SCORE: ', emo)    
print('AVERAGE GLOBAL SOCIAL SCORE: ', social)


AVERAGE GLOBAL TOTAL SCORE:  0.93

AVERAGE GLOBAL LOGICAL SCORE:  0.85
AVERAGE GLOBAL EMO SCORE:  0.96
AVERAGE GLOBAL SOCIAL SCORE:  0.97


In [187]:
dic_of_models = {m:{'total':[], 'logical':[], 'emotional':[], 'social':[]} for m in models}

for i,m in enumerate(dic_of_scores['model']):
    mod = dic_of_models[m]
    for key, val in mod.items():
        mod[key].append(dic_of_scores[key][i])


In [188]:
from scipy import stats
import itertools

items = list(itertools.combinations(list(dic_of_models[models[0]].keys()), 2))

for x, y in items:
    ref_data = [mean(dic_of_models[m][x]) for m in models]

    corres_data = [mean(dic_of_models[m][y]) for m in models]
    
    corr, _ = stats.pearsonr(ref_data, corres_data)
    print(x, y, round(corr,2))

total logical 0.8
total emotional 0.8
total social 0.01
logical emotional 0.35
logical social -0.5
emotional social 0.26


In [189]:
mtx_2 = np.zeros((len(models), len(dic_of_models[models[0]])))

for i, model in enumerate(models):
    for j, score in enumerate(list(dic_of_models[model].keys())):
        mtx_2[i][j] = round(mean(dic_of_models[model][score]),2)

In [190]:
panda_df = pd.DataFrame(data = mtx_2,  
                        index = models,  
                        columns = [x +' consistency score' for x in list(dic_of_models[model].keys())]) 
print(panda_df)

                        total consistency score  logical consistency score  \
qwen naive planning                        0.95                       0.87   
qwen response e2e                          0.91                       0.77   
llama naive planning                       0.93                       0.86   
llama response e2e                         0.93                       0.86   
mixtral naive planning                     0.92                       0.86   
mixtral response e2e                       0.90                       0.82   
ground truth                               0.94                       0.89   

                        emotional consistency score  social consistency score  
qwen naive planning                            1.00                      0.98  
qwen response e2e                              0.97                      0.99  
llama naive planning                           0.95                      0.97  
llama response e2e                             0.95    

In [191]:
for model in models:
    for k, v in dic_of_models[model].items():
        human_scores[model][k] = round(mean(v),2)

In [192]:
human_scores['ground truth']

{'usefulness': 0.87,
 'fluency': 0.92,
 'naturalness': 0.88,
 'emo f1': 0.68,
 'emotion_adequate': 0.98,
 'strategy_adequate': 0.9,
 'lev strat': 0.11,
 'jacc strat': 0.84,
 'role_consistency': 1,
 'total': 0.94,
 'logical': 0.89,
 'emotional': 0.98,
 'social': 0.95}

In [193]:
#SAVE IN TXT FILE
file_generated = "all_scores"
print('Saving results...')
with open("results/"+file_generated+".txt", "w") as f:
  print(panda_df, file=f)

Saving results...


<h1>Automatic Evaluation</h1>

In [203]:
models = [x for x in data_ref.columns if x not in ['context', 'emotions', 'strats', 'gt segments', 'gt segment strats']]
models

['qwen naive planning',
 'qwen response e2e',
 'qwen 2 step planning',
 'qwen 3 steps planning',
 'llama naive planning',
 'llama response e2e',
 'llama 2 step planning',
 'llama 3 steps planning',
 'mixtral naive planning',
 'mixtral response e2e',
 'mixtral 2 step planning',
 'mixtral 3 steps planning',
 'qwen dpo response e2e',
 'qwen dpo naive planning',
 'ground truth']

In [204]:
df_per_model = {m: list(data_ref[m]) for m in models}
df_per_model['context'] = list(data_ref['context'])
df_per_model['emotions'] = [ast.literal_eval(x) for x in list(data_ref['emotions'])]
df_per_model['strats'] = list(data_ref['strats'])

df_per_model['gt segments'] = [ast.literal_eval(x) for x in list(data_ref['gt segments'])]
df_per_model['gt segment strats'] = [ast.literal_eval(x) for x in list(data_ref['gt segment strats'])]

df_per_model['ground truth'] = [x[0].upper() + x[1:] for x in df_per_model['ground truth']]

In [205]:
len(df_per_model['context'])

213

<h2>Comparison-Based Metrics</h2>

In [206]:
#LOAD METRICS
import evaluate

sacrebleu = evaluate.load("sacrebleu")
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")
chrf = evaluate.load("chrf")
perplexity = evaluate.load("perplexity", module_type="metric")
meteor = evaluate.load("meteor")

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lorrainevanel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/lorrainevanel/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/lorrainevanel/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [210]:
metrics = ['bleu', 'rouge', 'bertscore', "chrf", 'bertscore_context', 'perplexity', 'meteor']

df_results = {model: {metric: [] for metric in metrics} for model in models}

In [211]:
#BERST SCORE CONTEXT - RESPONSE
contexts = [x.replace("Customer(C): ", "").replace("Agent(A): ", "")for x in df_per_model['context']]

In [212]:
# CLASSIC NLP REFERENCE COMPARISON AUTOMATED METRICS

for model in tqdm(models):
    print(model)
    if model != "ground truth":
        #BLEU
        bleu= sacrebleu.compute(predictions=df_per_model[model], references=df_per_model['ground truth'])
        df_results[model]['bleu'] = round(bleu['score'],2)
        
        #ROUGE
        rouge1 = rouge.compute(predictions=df_per_model[model], references=df_per_model['ground truth'])
        rouge1 = rouge1['rouge1']
        df_results[model]['rouge'] = round(rouge1,2)
        
        #BERT SCORE
        bs = bertscore.compute(predictions=df_per_model[model], references=df_per_model['ground truth'], lang='fr')
        f1 = bs['f1']
        avg_f1_bert = round(sum(f1) / len(f1),2)
        df_results[model]['bertscore'] = avg_f1_bert
        
        #CHRF
        df_results[model]['chrf'] = round(chrf.compute(predictions=df_per_model[model], references=df_per_model['ground truth'])['score'],2)

        #METEOR
        df_results[model]['meteor'] = round(meteor.compute(predictions=df_per_model[model], references=df_per_model['ground truth'])['meteor'], 2)
    
    #BERTSCRE RESPONSE / CONTEXT
    bs_c = bertscore.compute(predictions=df_per_model[model], references=contexts, lang='fr')
    f1_c = bs_c['f1']
    avg_f1_bert_c = round(sum(f1_c) / len(f1_c),2)
    df_results[model]['bertscore_context'] = f1_c

    #PERPLEXITY
    ppl = perplexity.compute(model_id='gpt2', predictions=df_per_model[model])
    df_results[model]['perplexity'] = ppl['perplexities']

  0%|          | 0/15 [00:00<?, ?it/s]

qwen naive planning


  7%|▋         | 1/15 [02:14<31:25, 134.65s/it]

qwen response e2e


 13%|█▎        | 2/15 [04:21<28:14, 130.31s/it]

...

 87%|████████▋ | 13/15 [46:11<11:54, 357.37s/it]

qwen dpo naive planning


 93%|█████████▎| 14/15 [55:48<07:03, 423.67s/it]

ground truth


100%|██████████| 15/15 [59:51<00:00, 239.46s/it]


In [213]:
# RESPONSE MEAN LENGTH
for model in models:
    df_results[model]['avg length char'] = [len(str(x)) for x in df_per_model[model]]
    df_results[model]['avg length words'] = [len(str(x).split(' ')) for x in df_per_model[model]]

In [214]:
from yanzhu_metrics import lexical_diversity, semantic_diversity

for model in models:
    df_results[model]['semantic diversity'] = float(semantic_diversity([x for x in df_per_model[model] if isinstance(x, str)]))
    df_results[model]['lexical diversity'] = lexical_diversity([x for x in df_per_model[model] if isinstance(x, str)])

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
df_results[models[0]].keys()

<h2>Socio-Emotional Criteria</h2>

In [215]:
def split_punct(text):

    punc = ['.', '!', '?']

    if isinstance(text, str):
        for p in punc:
            text = text.replace(p, p+'<SEP>')
        segs = text.split('<SEP>')

        temp = []
        for seg in segs:
            if len(seg.strip()) > 0:
                temp.append(seg.strip())
        
        return temp
    
    else:
        return np.nan

In [216]:
dic_segments = {m:{'response': df_per_model[m], 'full_annots': [], 'segments': [split_punct(x) for x in df_per_model[m]], 'annots':[], 'emotion': []} for m in models if m != 'ground truth'}

In [217]:
ref_emotions = []

for x in df_per_model['emotions']:
    if x == []:
        ref_emotions.append('neutre')
    
    else:
        ref_emotions.append(x[0].lower())

In [218]:
ref = {'ground truth': df_per_model['ground truth'], 'full_annots': df_per_model['strats'], 'segments': df_per_model['gt segments'], 'annots': df_per_model['gt segment strats'], 'emotion': ref_emotions}

<h3>Emotional Score</h3>

In [220]:
#Presence of Emotion = Classifier

from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import pandas as pd

model_name = "classifier/emo_class/checkpoint-40"
tokenizer = AutoTokenizer.from_pretrained(model_name) 
model = AutoModelForSequenceClassification.from_pretrained(model_name)
config = model.config


for m in dic_segments.keys():

    texts = dic_segments[m]['response']
    preds = []

    for text in tqdm(texts):
        if isinstance(text, str):
            inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512) 
            outputs = model(**inputs)

            prediction = torch.nn.functional.softmax(outputs.logits, dim=-1) 
            predicted_emotion = prediction.argmax().item()
            preds.append(config.id2label[predicted_emotion])
        else:
            preds.append('neutre')

    dic_segments[m]['emotion'] = preds

100%|██████████| 213/213 [00:13<00:00, 16.25it/s]


In [223]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

for m in dic_segments.keys():
    scores = precision_recall_fscore_support(ref['emotion'], dic_segments[m]['emotion'], average='weighted')

    print(m, ': Accuracy ', round(accuracy_score(ref['emotion'], dic_segments[m]['emotion']),2), ' - Precision ', round(scores[0], 2), ' - Recall : ', round(scores[1], 2), ' - F1 : ', round(scores[2], 2))

    df_results[m]['emotion f1'] = round(f1_score(ref['emotion'], dic_segments[m]['emotion'], average='weighted'),2)

qwen naive planning : Accuracy  0.66  - Precision  0.73  - Recall :  0.66  - F1 :  0.69
qwen response e2e : Accuracy  0.64  - Precision  0.67  - Recall :  0.64  - F1 :  0.65
qwen 2 step planning : Accuracy  0.58  - Precision  0.68  - Recall :  0.58  - F1 :  0.62
qwen 3 steps planning : Accuracy  0.56  - Precision  0.67  - Recall :  0.56  - F1 :  0.6
llama naive planning : Accuracy  0.67  - Precision  0.69  - Recall :  0.67  - F1 :  0.68
llama response e2e : Accuracy  0.61  - Precision  0.69  - Recall :  0.61  - F1 :  0.64
llama 2 step planning : Accuracy  0.58  - Precision  0.68  - Recall :  0.58  - F1 :  0.62
llama 3 steps planning : Accuracy  0.59  - Precision  0.69  - Recall :  0.59  - F1 :  0.63
mixtral naive planning : Accuracy  0.46  - Precision  0.69  - Recall :  0.46  - F1 :  0.53
mixtral response e2e : Accuracy  0.5  - Precision  0.67  - Recall :  0.5  - F1 :  0.55
mixtral 2 step planning : Accuracy  0.58  - Precision  0.7  - Recall :  0.58  - F1 :  0.63
mixtral 3 steps planni

/Users/lorrainevanel/anaconda3/envs/lo/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/lorrainevanel/anaconda3/envs/lo/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/lorrainevanel/anaconda3/envs/lo/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.

<h3>Social Score</h3>

In [226]:
# train model to classify dialogue strategies, compare sequences in predicted responses to ground truth, use levenshtein to compute a score.
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import pandas as pd
from tqdm import tqdm

model_name = "classifier/camembert-large/checkpoint-770"
tokenizer = AutoTokenizer.from_pretrained(model_name) 
model = AutoModelForSequenceClassification.from_pretrained(model_name)
config = model.config

for m in dic_segments.keys():

    texts = dic_segments[m]['segments']
    annots = []

    for segments in tqdm(texts):
        temp = []
        if isinstance(segments, list):
            for segment in segments:
                if isinstance(segment.strip(), str):

                    inputs = tokenizer(segment.strip(), return_tensors="pt", padding=True, truncation=True, max_length=512) 
                    outputs = model(**inputs)

                    prediction = torch.nn.functional.softmax(outputs.logits, dim=-1) 
                    predicted_emotion = prediction.argmax().item()
                    temp.append(config.id2label[predicted_emotion])

        annots.append(temp)

    dic_segments[m]['annots'] = annots
    dic_segments[m]['full_annots'] = ['+'.join(x) for x in annots]

SafetensorError: Error while deserializing header: MetadataIncompleteBuffer

In [227]:
from textdistance import levenshtein
import distance

for m in dic_segments.keys():
    lev = round(mean([levenshtein.normalized_similarity(ref['annots'][i], dic_segments[m]['annots'][i]) for i in range(len(dic_segments[m]['annots']))]),2)
    jacc = round(mean([distance.jaccard(set(ref['annots'][i]), set(dic_segments[m]['annots'][i])) for i in range(len(dic_segments[m]['annots']))]), 2)

    print(m, ': Levenshtein Similarity ', lev, ' - Jaccard Similarity ', jacc)
    df_results[m]['Levenshtein Similarity DS'] = lev
    df_results[m]['Jaccard Similarity DS'] = jacc

StatisticsError: mean requires at least one data point

<h2>Saving Results!</h2>

In [ ]:
import numpy as np

res_arry = np.zeros((len(models), len(df_results[models[0]].keys())))
scores_all = {m:{k:0 for k in df_results[models[0]].keys()} for m in models}


for i, model in enumerate(models):
    for j, metric in enumerate(df_results[model].keys()):

        v = df_results[model][metric]

        if v != []:        
            if isinstance(v, list):
                if metric == "score":
                    res_arry[i][j] = 1 - round(mean(v), 2)
                    scores_all[model][metric] =1 - round(mean(v), 2)

                else:
                    res_arry[i][j] = round(mean(v), 2)
                    scores_all[model][metric] = round(mean(v), 2)
                
            if isinstance(v, float):
                res_arry[i][j] = round(v,2)
                scores_all[model][metric] = round(v,2)

            elif isinstance(v, int):
                res_arry[i][j] = v
                scores_all[model][metric] = v

        elif metric == "semantic similarity":
            res_arry[i][j] = round(v,2)
            scores_all[model][metric] = res_arry[i][j]

        elif metric == "rouge" and model != "ground truth":
            res_arry[i][j] = v
            scores_all[model][metric] = v

        elif metric == "meteor" and model != "ground truth":
            res_arry[i][j] = v
            scores_all[model][metric] = v

        else:
            res_arry[i][j] = None
            scores_all[model][metric] = res_arry[i][j]


In [ ]:
panda_df = pd.DataFrame(data = res_arry,  
                        index = models,  
                        columns = df_results[models[0]].keys()) 
print(panda_df)

In [ ]:
#SAVE IN TXT FILE
file_generated = "all_results"
print('Saving results...')
with open("results/"+file_generated+".txt", "w") as f:
  print(panda_df, file=f)

<h1>Pearson Correlation between Human Evaluation and Automatic Metrics</h1>

In [ ]:
all_human_annots = human_scores

for k, v in all_human_annots.items():
    for a, b in filter_scores[k].items():
        all_human_annots[k][a] = b

In [ ]:
correspondance = {'filtered weighted': 'bertscore_context', 'fluency': ['perplexity'], "naturalness": 'semantic diversity', "emotion_adequate": 'emotion f1', 'strategy_adequate': 'Levenshtein Similarity DS'}

In [ ]:
from scipy import stats

correl_scores= {k:0 for k in list(correspondance.keys())}

for ref, corres in correspondance.items():
    if isinstance(corres, list):
        temp = []
        for metric in corres:
            ref_data = [human_scores[m][ref] for m in list(human_scores.keys())if m != "ground truth"]
            #print(ref_data)

            corres_data = [scores_all[m][metric] for m in list(human_scores.keys()) if m != "ground truth"]
            #print(corres_data)

            corr, _ = stats.pearsonr(ref_data, corres_data)
            temp.append(corr)
        correl_scores[ref] = round(mean(temp),2)
    else:
        ref_data = [human_scores[m][ref] for m in list(human_scores.keys())if m != "ground truth"]
        #print(ref_data)

        corres_data = [scores_all[m][corres] for m in list(human_scores.keys())if m != "ground truth"]
        #print(corres_data)        
        
        corr, _ = stats.pearsonr(ref_data, corres_data)
        correl_scores[ref] = round(corr,2)

for k, v in correl_scores.items():
    print(k, float(v))

<h1>Strategy Specific Metrics</h1>

In [ ]:
#DEFINE STRATEGIES
strategies_dic = {'HS': 'Hors-sujet',
       "A": 'Accord',
       "AGR": 'Agressivité',
       "BC": "Back-channeling",
       "C": 'Correction',
       "D": "Désaccord",
       "E": "Encouragement", 
       "I": "Information",
       "P": "Politesse",
       "PS": "Propositiondesuggestion",
       "Q": "Question",
       "R": "Reformulation",
       "S": "Sympathie",
       "SD": "Self-disclosure",
       "U": "Autre"}

strategies = list(strategies_dic.keys())
strategies

In [ ]:
#for each class use yanzhu's metrics  - Generated AND Reference
from yanzhu_metrics import lexical_diversity, semantic_diversity

yanzhu_strats = {m: {k:{'semantic':0, 'lexical':0, 'len': 0} for k in strategies} for m in models}

for m in models:
    print(m)


    for strat in tqdm(strategies):

        sentences_strat = []
        if m == "ground truth":
             a, b = ref['segments'], ref['annots']

        else:
             a, b = dic_segments[m]['segments'], dic_segments[m]['annots']

        for segment, annot in zip(a,b):
            if strat in annot:
                sentences_temp = [x for j,x in enumerate(segment) if annot[j] == strat]
                for s in sentences_temp:
                        if len(s) > 1:
                            sentences_strat.append(s)

        if len(sentences_strat) > 0:
            yanzhu_strats[m][strat]['len'] = len(sentences_strat)
            yanzhu_strats[m][strat]['semantic'] = lexical_diversity(sentences_strat)
            yanzhu_strats[m][strat]['lexical'] = semantic_diversity(sentences_strat)

In [ ]:
mtx_strat= np.zeros((len(models), len(strategies)))
for k, v in yanzhu_strats.items():
    for strat, value in v.items():
        print(k,'\t', strat, '\t', value['len'], '\t', round(value['semantic'],2), '\t', round(value['lexical'],2))

In [ ]:
mtx_strat= np.zeros((len(models)*2, len(strategies)))

i =0
for model in models:
    for j, strat in enumerate(strategies):
        mtx_strat[i][j] = int(yanzhu_strats[model][strat]['semantic'])
        mtx_strat[i+1][j] = int(yanzhu_strats[model][strat]['len'])

    i+= 2

In [ ]:
mtx_lex= np.zeros((len(models)*2, len(strategies)))

i =0
for model in models:
    for j, strat in enumerate(strategies):
        mtx_lex[i][j] = round(yanzhu_strats[model][strat]['lexical'],2)
        mtx_lex[i+1][j] = int(yanzhu_strats[model][strat]['len'])

    i+= 2

In [ ]:
cols = []

for m in models:
    cols.append(m)
    cols.append(m + ' len')

In [ ]:
panda_df = pd.DataFrame(mtx_strat,  
                        index = cols,  
                        columns = strategies) 
print(panda_df)

#SAVE IN TXT FILE
file_generated = "yanzhu_sem"
print('Saving results...')
with open("results/"+file_generated+".txt", "w") as f:
  print(panda_df, file=f)

In [ ]:
panda_df = pd.DataFrame(mtx_lex,  
                        index = cols,  
                        columns = strategies) 
print(panda_df)

#SAVE IN TXT FILE
file_generated = "yanzhu_lex"
print('Saving results...')
with open("results/"+file_generated+".txt", "w") as f:
  print(panda_df, file=f)

In [ ]:
#For each strat, get avg for context & segment - Generated AND Reference 
#especially for reformulation
bert_strats = {m:{k:{'len': 0, 'bertscore': 0}  for k in strategies} for m in models}


for m in models:
    print(m)

    for strat in tqdm(strategies):

        sentences_strat = []
        if m == "ground truth":
             a, b = ref['segments'], ref['annots']

        else:
             a, b = dic_segments[m]['segments'], dic_segments[m]['annots']

        sentences, context_sentences = [], [] 

        i = 0
        for segment, annot in zip(a,b):
            if strat in annot:
                sentences_temp = [x for j,x in enumerate(segment) if annot[j] == strat]
                for s in sentences_temp:
                    sentences.append(s)
                    context_sentences.append(df_per_model['context'][i])
            i +=1
        
        if len(sentences) > 0:
            bert_score_strat = bertscore.compute(predictions=sentences, references=context_sentences, lang='fr')
            f1_strat = bert_score_strat['f1']
            bert_strats[m][strat]['len'] = len(sentences)
            bert_strats[m][strat]['bertscore'] = sum(f1_strat) / len(f1_strat)

In [ ]:
mtx_bert= np.zeros((len(models)*2, len(strategies)))

i =0
for model in models:
    for j, strat in enumerate(strategies):
        if bert_strats[model][strat]['bertscore'] > 0:
            mtx_bert[i][j] = round(bert_strats[model][strat]['bertscore'],2)
            mtx_bert[i+1][j] = int(bert_strats[model][strat]['len'])

    i+= 2

In [ ]:
panda_df = pd.DataFrame(mtx_bert,  
                        index = cols,  
                        columns = strategies) 
print(panda_df)

#SAVE IN TXT FILE
file_generated = "bert_strat"
print('Saving results...')
with open("results/"+file_generated+".txt", "w") as f:
  print(panda_df, file=f)

In [ ]:
for k, v in bert_strats.items():
    for strat, value in v.items():
        if value['len'] >0:
            print(k,' - ', strat, ' - len: ', value['len'], ' bertscore : ', value['bertscore'])